In [2]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import os

In [26]:
import mle.tsa as mle_tsa

Importing plotly failed. Interactive plots will not work.


In [16]:
OUTPATH = 'data/out/'

In [23]:
outfiles = os.listdir(OUTPATH)
outfiles

['cons_gna95_test_fbprophet.xlsx',
 'cons_gna95_test_naive.xlsx',
 'cons_gna95_test_sarimax.xlsx',
 'cons_gna95_test_var.xlsx']

In [24]:
test_preds = []
for file in outfiles:
    print(file)
    df = pd.read_excel(OUTPATH + file, parse_dates=['Date'], index_col='Date')
    test_preds.append(df)

p_gna95_test = pd.concat(test_preds, axis=0, sort=False)

cons_gna95_test_fbprophet.xlsx
cons_gna95_test_naive.xlsx
cons_gna95_test_sarimax.xlsx
cons_gna95_test_var.xlsx


In [30]:
p_gna95_test['ape_cons_GNA95'] = p_gna95_test[['cons_GNA95', 'p_cons_GNA95']].apply(lambda x: mle_tsa.compute_ape(y_true=x[0], y_pred=x[1]), axis=1)

In [31]:
p_gna95_test.groupby('model')['ape_cons_GNA95'].mean()

model
fbprophet    0.013020
naive        0.067712
sarimax      0.040938
var          0.025955
Name: ape_cons_GNA95, dtype: float64